加噪声测试

已完成：调整学习率，降低代码复杂度

In [ ]:
import tqdm
from torch.nn.utils import clip_grad_norm_
from pcdet.models_multinomial_half.detectors.pointpillar import PointPillar

import argparse
import datetime
import glob
import os
from pathlib import Path
from test import repeat_eval_ckpt, eval_single_ckpt
# from noise import add_noise_to_weights
from hardware_noise.weight_mapping_finalv import weight_mapping as add_noise_to_weights

import numba
import logging
import time

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

import torch
import torch.distributed as dist
import torch.nn as nn
from tensorboardX import SummaryWriter

from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.datasets import build_dataloader
from pcdet.models_multinomial_half import build_network, model_fn_decorator
from pcdet.utils import common_utils
from train_utils.optimization import build_optimizer, build_scheduler
# from train_utils.train_utils import train_model
# from eval_utils import eval_utils_multinomial
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
from eval_utils import eval_utils_multinomial_finalv_half as eval_utils

import warnings
warnings.filterwarnings("ignore")


def load_data_to_gpu(batch_dict):
    for key, val in batch_dict.items():
        if not isinstance(val, np.ndarray):
            continue
        elif key in ['frame_id', 'metadata', 'calib']:
            continue
        elif key in ['images']:
            batch_dict[key] = image_to_tensor(val).float().cuda().contiguous()
        elif key in ['image_shape']:
            batch_dict[key] = torch.from_numpy(val).int().cuda()
        else:
            batch_dict[key] = torch.from_numpy(val).float().cuda()


def parse_config():
    parser = argparse.ArgumentParser(description='arg parser')
    parser.add_argument('--cfg_file', type=str, default='./cfgs/kitti_models/pointpillar_bayes.yaml', help='specify the config for training')
    parser.add_argument('--batch_size', type=int, default=8, required=False, help='batch size for training')
    parser.add_argument('--workers', type=int, default=32, help='number of workers for dataloader')
    parser.add_argument('--extra_tag', type=str, default='default', help='extra tag for this experiment')
    parser.add_argument('--pretrained_model', type=str, default=True, help='pretrained_model')
    parser.add_argument('--launcher', choices=['none', 'pytorch', 'slurm'], default='none')
    parser.add_argument('--tcp_port', type=int, default=18888, help='tcp port for distrbuted training')
    parser.add_argument('--sync_bn', action='store_true', default=False, help='whether to use sync bn')
    parser.add_argument('--fix_random_seed', action='store_true', default=True, help='')
    parser.add_argument('--ckpt_save_interval', type=int, default=80, help='number of training epochs')
    parser.add_argument('--local_rank', type=int, default=0, help='local rank for distributed training')
    parser.add_argument('--max_ckpt_save_num', type=int, default=81, help='max number of saved checkpoint')
    parser.add_argument('--merge_all_iters_to_one_epoch', action='store_true', default=False, help='')
    parser.add_argument('--set', dest='set_cfgs', default=None, nargs=argparse.REMAINDER, help='set extra config keys if needed')
    parser.add_argument('--max_waiting_mins', type=int, default=0, help='max waiting minutes')
    parser.add_argument('--start_epoch', type=int, default=0, help='')
    parser.add_argument('--save_to_file', action='store_true', default=False, help='')

    parser.add_argument('--epochs', type=int, default=800, required=False, help='number of epochs to train for')
    parser.add_argument('--ckpt', type=str, default='checkpoint_epoch_80_multinomial.pth', help='checkpoint to start from')
    # sunqiao/OpenPCDet/tools/save_path/0722/ckpt_epoch84.pth
    args = parser.parse_known_args()[0]

    cfg_from_yaml_file(args.cfg_file, cfg)
    cfg.TAG = Path(args.cfg_file).stem
    cfg.EXP_GROUP_PATH = '/'.join(args.cfg_file.split('/')[1: -1])  # remove 'cfgs' and 'xxxx.yaml'

    if args.set_cfgs is not None:
        cfg_from_list(args.set_cfgs, cfg)

    return args, cfg




if __name__ == '__main__':
    torch.cuda.set_device(0)

    args, cfg = parse_config()
    dist_train = False
    total_gpus = 1
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
    memory_gpu = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
    print('Using GPU:' + str(np.argmax(memory_gpu)))
    os.environ["CUDA_VISIBLE_DEVICES"] = str(np.argmax(memory_gpu))
    os.system('rm tmp')


    if args.batch_size is None:
        args.batch_size = cfg.OPTIMIZATION.BATCH_SIZE_PER_GPU
    else:
        assert args.batch_size % total_gpus == 0, 'Batch size should match the number of gpus'
        args.batch_size = args.batch_size // total_gpus

    args.epochs = cfg.OPTIMIZATION.NUM_EPOCHS if args.epochs is None else args.epochs
    
    common_utils.set_random_seed(666)
    save_path = './save_path/logger/'#/bayes/pointpillar/'+time.strftime('%m%d-%H%M',time.localtime(time.time()))+'/'

    logger = common_utils.create_logger(save_path+'log.txt', rank=cfg.LOCAL_RANK)
    
    print("=============")
    p1 = 0.5766
    p2 = 0.8603
    p3 = 0.5753
    print(p1, p2, p3)
    print("=============")

    # train_set, train_loader, train_sampler = build_dataloader(
    #     dataset_cfg=cfg.DATA_CONFIG,
    #     class_names=cfg.CLASS_NAMES,
    #     batch_size=args.batch_size,
    #     dist=dist_train, workers=args.workers,
    #     logger=logger,
    #     training=True,
    #     merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
    #     total_epochs=args.epochs
    # )

    test_set, test_loader, sampler = build_dataloader(
                                    dataset_cfg=cfg.DATA_CONFIG,
                                    class_names=cfg.CLASS_NAMES,
                                    batch_size=args.batch_size,
                                    dist=dist_train, workers=args.workers, logger=logger, training=False
    )

    model = PointPillar(
        model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), p1=p1, p2=p2, p3=p3, dataset=test_set
    )

    model.load_state_dict(torch.load(args.ckpt, map_location=torch.device('cuda:0'))['model_state'])
    # torch.load('./checkpoint_epoch_33_multinomial.pth')['model_state']

    model.cuda()

    # hw_data_files = sorted(os.listdir('./hardware_noise/hardware_data/'))
    file2ap_dict = {}
    N = 10

    f_dir = './hardware_noise/hardware_IV_and_condition_summary/hardware_data/'
    file2usability = np.load('./hardware_noise/file2usability.npy', allow_pickle=True).item()
    file2sigma = np.load('./hardware_noise/file2sigma.npy', allow_pickle=True).item()

    # for f_name in sorted(os.listdir(f_dir)):
    for sigma, f_name in sorted(zip(file2sigma.values(), file2sigma.keys())):
        if not f_name.endswith('.xlsx'):
            continue
        # print(f_name)
        usability = file2usability[f_name]
        # sigma = file2sigma[f_name]
        file2ap_dict[f_name] = {}
        for n in range(N):
            print('file: {}, sigma: {}, usability: {}, evaluate-{}'.format(f_name, sigma, usability, n))
            model.load_state_dict(torch.load(args.ckpt, map_location=torch.device('cuda:0'))['model_state'])
            add_noise_to_weights(f_dir+f_name, model, noise_sigma=sigma, device='cuda')

            acc1, ret_dict = eval_utils.eval_simple(
                p1, p2, p3, f_name, usability, sigma, n, cfg, model, test_loader, logger, 
                save_path=None, dist_test=dist_train,
                save_to_file=args.save_to_file, result_dir=None
            )
            print(ret_dict)
            file2ap_dict[f_name][n] = ret_dict

    logger.info('**********************End evaluation**********************')    



2023-07-27 12:30:04,034   INFO  Loading KITTI dataset
2023-07-27 12:30:04,131   INFO  Total samples for KITTI dataset: 3769


Using GPU:0
0.5766 0.8603 0.5753
file: I-V_data_1.5um_length_200nm_diameter_BT_4V_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_8V_carbon_paste_no_wait_2.xlsx, sigma: 0.06283342003106338, usability: 0.25709728349414823, evaluate-0


eval: 100%|██████████| 472/472 [03:45<00:00,  2.09it/s]


{'recall/roi_0.3': 0.0, 'recall/rcnn_0.3': 0.0, 'recall/roi_0.5': 0.0, 'recall/rcnn_0.5': 0.0, 'recall/roi_0.7': 0.0, 'recall/rcnn_0.7': 0.0, 'Car_aos/easy_R40': 96.69680077023854, 'Car_aos/moderate_R40': 88.86978426421355, 'Car_aos/hard_R40': 87.49752080655506, 'Car_3d/easy_R40': 87.40340980750744, 'Car_3d/moderate_R40': 73.76802461364063, 'Car_3d/hard_R40': 70.71028418357668, 'Car_bev/easy_R40': 93.06047840168968, 'Car_bev/moderate_R40': 85.28847284325215, 'Car_bev/hard_R40': 84.01162622979841, 'Car_image/easy_R40': 96.72892903340596, 'Car_image/moderate_R40': 89.10679229279482, 'Car_image/hard_R40': 87.90448855486073, 'Pedestrian_aos/easy_R40': 33.327541164851475, 'Pedestrian_aos/moderate_R40': 29.14851168944598, 'Pedestrian_aos/hard_R40': 26.594238951108697, 'Pedestrian_3d/easy_R40': 36.279385294841966, 'Pedestrian_3d/moderate_R40': 30.225204028893664, 'Pedestrian_3d/hard_R40': 26.20910470099222, 'Pedestrian_bev/easy_R40': 54.005140721938226, 'Pedestrian_bev/moderate_R40': 46.38458

eval: 100%|██████████| 472/472 [03:40<00:00,  2.14it/s]


...

eval: 100%|██████████| 472/472 [06:06<00:00,  1.29it/s]


{'recall/roi_0.3': 0.0, 'recall/rcnn_0.3': 0.0, 'recall/roi_0.5': 0.0, 'recall/rcnn_0.5': 0.0, 'recall/roi_0.7': 0.0, 'recall/rcnn_0.7': 0.0, 'Car_aos/easy_R40': 0.015928177759496856, 'Car_aos/moderate_R40': 0.02909437992850864, 'Car_aos/hard_R40': 0.03216940186301237, 'Car_3d/easy_R40': 0.0001067187621628001, 'Car_3d/moderate_R40': 0.00012675328413670381, 'Car_3d/hard_R40': 0.0001390190488807331, 'Car_bev/easy_R40': 0.002787626422034292, 'Car_bev/moderate_R40': 0.0024093706973379596, 'Car_bev/hard_R40': 0.0011974418731575077, 'Car_image/easy_R40': 0.04076246489252411, 'Car_image/moderate_R40': 0.07130635304974149, 'Car_image/hard_R40': 0.07377689827825186, 'Pedestrian_aos/easy_R40': 0.0, 'Pedestrian_aos/moderate_R40': 0.0, 'Pedestrian_aos/hard_R40': 0.0, 'Pedestrian_3d/easy_R40': 0.0, 'Pedestrian_3d/moderate_R40': 0.0, 'Pedestrian_3d/hard_R40': 0.0, 'Pedestrian_bev/easy_R40': 0.0, 'Pedestrian_bev/moderate_R40': 0.0, 'Pedestrian_bev/hard_R40': 0.0, 'Pedestrian_image/easy_R40': 0.0, 'Pe

eval: 100%|██████████| 472/472 [06:17<00:00,  1.25it/s]


{'recall/roi_0.3': 0.0, 'recall/rcnn_0.3': 0.0, 'recall/roi_0.5': 0.0, 'recall/rcnn_0.5': 0.0, 'recall/roi_0.7': 0.0, 'recall/rcnn_0.7': 0.0, 'Car_aos/easy_R40': 0.0031581310553270405, 'Car_aos/moderate_R40': 0.008853658305507685, 'Car_aos/hard_R40': 0.011349530703813011, 'Car_3d/easy_R40': 9.646313529743969e-05, 'Car_3d/moderate_R40': 8.910142376327223e-05, 'Car_3d/hard_R40': 0.00012783919224225294, 'Car_bev/easy_R40': 0.00032556421034779347, 'Car_bev/moderate_R40': 0.0003842541200580651, 'Car_bev/hard_R40': 0.0005451459677588852, 'Car_image/easy_R40': 0.0069707446568180904, 'Car_image/moderate_R40': 0.01768206142140513, 'Car_image/hard_R40': 0.024617722180212173, 'Pedestrian_aos/easy_R40': 0.0, 'Pedestrian_aos/moderate_R40': 0.0, 'Pedestrian_aos/hard_R40': 0.0, 'Pedestrian_3d/easy_R40': 0.0, 'Pedestrian_3d/moderate_R40': 0.0, 'Pedestrian_3d/hard_R40': 0.0, 'Pedestrian_bev/easy_R40': 0.0, 'Pedestrian_bev/moderate_R40': 0.0, 'Pedestrian_bev/hard_R40': 0.0, 'Pedestrian_image/easy_R40': 

eval:  66%|██████▌   | 311/472 [03:05<01:22,  1.96it/s]